# Integration over multiple level sets


The `ngsxfem` add-on library is capable of integrating over domians defined by muliple level set functions.

Previously, our doain of interest was described by a sige level set function, i.e., for a continuous level set function $\phi$, we considered the domain $\Omega := \{ \phi < 0 \}$.

Now, let $\{\phi_i \}$ be a set of level set functions. We define the domain of interest by the intersection $\Omega := \bigcap_i \{\phi_i < 0\}$.


As an example, we shall consider an isosceles triangle, described by the level set functions $\phi_1 = y - 1$, $\phi_2 = 2x - y$ and $\phi_3 = -2x - y$. 

The libraries are importes as usual:

In [ ]:
# Visualisation
import netgen.gui

# Import geometry features, NGSolve and xfem
from netgen.geom2d import SplineGeometry
from ngsolve import *
from xfem import *

# Visualisation properties
from ngsolve.internal import *

We generate a background mesh

In [ ]:
geo = SplineGeometry()
geo.AddRectangle([-1, -0.5], [1, 1.5], bc=1)
mesh = Mesh (geo.GenerateMesh(maxh=0.7, quad_dominated=False))

As in the single level set setting, `ngsxfem` can only handle piecewise linear level set functions. We thefore interpolate each smooth level set into an P1 GridFunction $\phi^\ast_i \in \mathcal{P}^1(T), T \subset \Omega$.

In [ ]:
level_sets = (y-1, 2*x-y, -2*x-y)
nr_ls = len(level_sets)
level_sets_p1 = tuple(GridFunction(H1(mesh, order=1)) for i in range(nr_ls))

lset_mult = CoefficientFunction(1)
for i, lset_p1 in enumerate(level_sets_p1):
    InterpolateToP1(level_sets[i], lset_p1)
    Draw(lset_p1, mesh, "lset_p1_{}".format(i))
    lset_mult *= IfPos(lset_p1, 0, 1)

Draw(lset_mult, mesh, "lset_mult", sd=7)

visoptions.mminval = 0.0
visoptions.mmaxval = 0.0
visoptions.deformation = 0
visoptions.autoscale = 0

Note: The fuzzy edges of the triangle in `lset_mult` is simply a graphical representation issue.

To integrate, we use the `Integrate` function as before, the difference being in the `levelset_domain` dictionary which we pass to the function. The `levelset_domain` dictionary must contain the two necessary keys `levelset` and `domain_type`. 
* The entry for `levelset` must be a `tuple` of the P1-GridFunction level set functions.
* The entry for `domain_type`  describes the domain with respect to each level set. A its most basic form, it is either a `tuple({NEG,POS,IF})` which describes the domain whith respect each level set function or a `list(tuple({NEG,POS,IF}))` of such tuples, if the domain is described via mulple regions.

In [ ]:
area = Integrate(levelset_domain={"levelset": level_sets_p1, "domain_type": (NEG, NEG, NEG)},
                     cf=1, mesh=mesh, order=0)
error = abs(area - 0.5)
print("Result of the integration: {}".format(area))
print("Error of the integration: {:5.3e}".format(error))

Since the smooth level set functions are linear, the integration is exact upto mashine precision.

### Integrating over objects of higher codimension

We can also integrate over subdomains of higher codimensions. For example, the lenghths of the sides of the triangle can be computed as

In [ ]:
len_top = Integrate(levelset_domain={"levelset": level_sets_p1, "domain_type": (IF, NEG, NEG)},
                     cf=1, mesh=mesh, order=0)
print("Top side-length of the triangle")
print("Result of the integration: {}".format(len_top))
print("Error of the integration: {:5.3e}".format(abs(len_top - 1)))

len_right = Integrate(levelset_domain={"levelset": level_sets_p1, "domain_type": (NEG, IF, NEG)},
                      cf=1, mesh=mesh, order=0)
print("\nRight side-length of the triangle")
print("Result of the integration: {}".format(len_right))
print("Error of the integration: {:5.3e}".format(abs(len_right - sqrt(5)/2)))

len_left = Integrate(levelset_domain={"levelset": level_sets_p1, "domain_type": (NEG, NEG, IF)},
                     cf=1, mesh=mesh, order=0)
print("\nLeft side-length of the triangle")
print("Result of the integration: {}".format(len_left))
print("Error of the integration: {:5.3e}".format(abs(len_left - sqrt(5)/2)))


Top compute the perimeter of the triangle in one go, we can pass a list of each side to `Integrate` via the `domain_type` entru in the `levelset_domain` dictionary: 

In [ ]:
len_perimeter = Integrate(levelset_domain={"levelset": level_sets_p1, 
                                           "domain_type": [(IF, NEG, NEG), (NEG, IF, NEG), (NEG, NEG, IF)]},
                          cf=1, mesh=mesh, order=0)
print("Perimeter of the triangle")
print("Result of the integration: {}".format(len_perimeter))
print("Error of the integration: {:5.3e}".format(abs(len_perimeter - 1 - sqrt(5))))

Similarly, we can also perform point evaluations at the intersections of the level sets

In [ ]:
point_val = Integrate(levelset_domain={"levelset" : level_sets_p1, "domain_type": (IF, IF, NEG)},
                      cf=x**2 + y**2, mesh=mesh, order=0)

print("Result of the integration: {}".format(point_val))
print("Error of the integration: {:5.3e}".format(abs(point_val - 1.25)))

## Convinience Layer

The python library `xfem.mlset` provides a convinence layer for working with domains defined via multiple level set functions. 